In [ ]:
import pandas as pd
from tqdm.notebook import tqdm, trange
import datetime
import numpy as np
import random

In [ ]:
def fix_time(time):
    return datetime.datetime.fromisoformat(time)

df = pd.read_csv('BPI_2012.csv')
df = df.rename(columns={'Unnamed: 0': 'case_id', 'Unnamed: 1': 'step_number'})
df['time:timestamp'] = df['time:timestamp'].apply(fix_time)
    
df_attr = pd.read_csv('BPI_attr_2012.csv')
df_attr = df_attr.rename(columns={'Unnamed: 0': 'case_id'})
df_attr['REG_DATE'] = df_attr['REG_DATE'].apply(fix_time)

In [ ]:
df = df[df['lifecycle:transition'] == 'COMPLETE']
df = df.reset_index(drop = True)

In [ ]:
df['time:time_between'] = df['time:timestamp'].diff()
df.loc[df['step_number'] == 0, 'time:time_between'] = pd.Timedelta(0)

df['time:weekday'] = [x.weekday() for x in df['time:timestamp']]
df['time:hour'] = [x.hour for x in df['time:timestamp']]

In [ ]:
#amount of total data that is training data:
train_data = 0.8
#amount of train data that is validation data:
validation_data = 0.1

In [ ]:
limit_date = df[df['case_id'] == round(df.iloc[-1, 0]*train_data)].iloc[0]['time:timestamp']
#limit_date = datetime.datetime(2012, 2, 3, 1, 1, 1, 633000, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))

In [ ]:
#training data_set
df_train = df
listo = []
for i in tqdm(range(0, 13087)):
    if df[df['case_id'] == i].iloc[-1]['time:timestamp'] > limit_date:
        listo.append(i)

for i in tqdm(range(0, len(listo))):
    df_train = df_train.drop(df_train[df_train['case_id']==listo[i]].index)

df_train = df_train.reset_index(drop = True)

In [ ]:
#testing data_set
mask = np.random.rand(len(df_train)) < validation_data
df_validation = df_train[mask]
df_train = df_train[~mask]

In [ ]:
#testing data_set
df_test = pd.DataFrame(columns = (df.columns))
listo = []
for i in tqdm(range(0, 13087)):
    if df[df['case_id'] == i].iloc[0]['time:timestamp'] > limit_date:
        listo.append(i)

for i in tqdm(range(0, len(listo))):
    df_test = df_test.append(df[df['case_id'] == listo[i]])

df_test = df_test.reset_index(drop = True)

In [ ]:
print(len(df_train))
print(len(df_validation))
print(len(df_test))

In [ ]:
df_train.to_csv('BPI_2012_train', index=False)

In [ ]:
df_validation.to_csv('BPI_2012_validation')

In [ ]:
df_test.to_csv('BPI_2012_test', index=False)

In [ ]:
def fix_time(time):
    return datetime.datetime.fromisoformat(time)

df_train = pd.read_csv('BPI_2012_train')
df_train['time:timestamp'] = df_train['time:timestamp'].apply(fix_time)
df_train['time:time_between'] = df_train['time:timestamp'].diff()
df_train.loc[df['step_number'] == 0, 'time:time_between'] = pd.Timedelta(0)

df_test = pd.read_csv('BPI_2012_test')
df_test['time:timestamp'] = df_test['time:timestamp'].apply(fix_time)
df_test['time:time_between'] = df_test['time:timestamp'].diff()
df_test.loc[df['step_number'] == 0, 'time:time_between'] = pd.Timedelta(0)

In [ ]:
def creating_dict_for_next_step_stats (df : pd.DataFrame, concept_name : str) -> dict:
    '''For an input action checks for all the possible next actions and counts their occurence'''
    
    dic_occurrence = {}
    dic_total_time = {}
    ids = list(df['case_id']) + ['editor: last id'] #Otherwise we check i+1-th position that does not exist
    times = list(df['time:time_between']) + [pd.Timedelta(0)] #Otherwise we check i+1-th position that does not exist
    names = df['concept:name']
    df_concept = df[names == concept_name]
    
    for i, row in df_concept.iterrows():
        if (ids[i] == ids[i+1]): #an instance of the same case
            if (names[i+1] not in dic_occurrence):
                dic_occurrence[names[i+1]] = 1
                dic_total_time[names[i+1]] = times[i+1]
            else:
                dic_occurrence[names[i+1]] += 1
                dic_total_time[names[i+1]] += times[i+1]
        else: #the last instance of the case
            if ('editor: close_case' not in dic_occurrence):
                dic_occurrence['editor: close_case'] = 1
                dic_total_time['editor: close_case'] = times[i+1]
            else:
                dic_occurrence['editor: close_case'] += 1
                dic_total_time['editor: close_case'] += times[i+1]
    
    #Compute average time
    dic_avg_time = {}
    for key in dic_total_time:
        dic_avg_time[key] = dic_total_time[key] / dic_occurrence[key]
        
    return(dic_occurrence, dic_avg_time)

In [ ]:
def choosing_next_action(dic : dict):
    '''Finds the max value of the input dict and returns the key of the max value'''
    
    max_key = max(dic, key=dic.get)
    return(max_key)

In [ ]:
def cycles_shortcut(actions : list, concept_name : str, max_length : int) -> list or bool:
    '''For saving the operating time, we will try to terminate the baseline early if we get into a loop
    max_length is the longest_trace parameter'''
    
    if(concept_name in actions): #the action has already been done
        if(actions[-1] == concept_name): #and it's the most recent action (self-loop)
            while(len(actions) < max_length): #filling the rest of the list with the current action if we're in a self-loop
                actions.append(concept_name)
        
        else: #it is not the most recent action
            placement = actions.index(concept_name) #locating the index of the "duplicate"
            aid_array = actions[placement:] #copying the values
            print("aid_array = ", aid_array)
            
            actions = actions + [0] * (max_length-len(actions)) #making [x, y, z, x] into [x, y, z, x, 0, 0, 0, ...]
            print("actions = ", actions)
            
            for i in range(placement+1, max_length): #iterating only over all the indices of 0's in actions
                actions[i] = aid_array[(i-placement)%len(aid_array)] #copying the list's values over and over again
        
        return(actions) #This return has to be then the return of the iterated_expected_actions
    
    else:
        return(False)

In [ ]:
def iterating_expected_actions(df : pd.DataFrame, concept_name : str, n : int) -> list:
    '''concept_name is the starting point (first action)
    n is the length of the longest trace ever observed
    It is stored in lonest_trace but for runtime reasons, use n so far'''
    
    longest_trace = max(df['step_number']) #finding the longest trace in the database (nr of steps)
    #note that we determine this AFTER deleting some rows with uncomplete steps. We should be running this on full df
    
    i = 0
    actions = [concept_name] #Here is the list that will store all the subsequent actions the algorithm decices to perform
    while (i < n): #terminate if we are exceeding the max number of steps
        wow = creating_dict_for_next_step_stats(df, concept_name)[0] #list all possible options
        concept_name = choosing_next_action(wow) #Choose the most commonly used option
        
        if(cycles_shortcut(actions, concept_name, n) != False): #Checks if we are stuck in a loop
            print("we are stuck in a loop")
            return(cycles_shortcut(actions, concept_name, n))
        
        if(concept_name == 'editor: close_case'): #If it is the "terminate" option - terminate
            break
        actions.append(concept_name) #Add the action to the list
        i += 1
    
    actions.append('editor: close_case')
    print('i = ', i, "n = ", n)
    
    return(actions)

In [ ]:
def add_expected_events(df : pd.DataFrame) -> list:
    all_events = df['concept:name'].unique()
    next_event_name_dic = {'editor: close_case': 'editor: close_case'}
    next_event_duration_dic = {'editor: close_case': pd.Timedelta(0)}
    for i in tqdm(range(0, len(all_events))):
        event = all_events[i]
        next_step_stats = creating_dict_for_next_step_stats(df, event)
        wow = next_step_stats[0] #list all possible options
        concept_name = choosing_next_action(wow) #Choose the most commonly used option
        next_event_name_dic[event] = concept_name
        next_event_duration_dic[event] = next_step_stats[1][concept_name]
    return next_event_name_dic, next_event_duration_dic

In [ ]:
all_expected_events = add_expected_events(df_train)
all_expected_events

In [ ]:
def prediction(prev):
    return all_expected_events[0][prev]

df_test['predicted'] = df_test['concept:name'].apply(prediction)

In [ ]:
df_test['correct'] = np.nan
for i in tqdm(range(0, len(df_test)-1)):
    if df_test.loc[i, 'case_id'] == df_test.loc[i+1, 'case_id']:
        df_test.loc[i, 'correct'] = (df_test.loc[i, 'predicted'] == df_test.loc[i+1, 'concept:name'])
    
    else:
         df_test.loc[i, 'correct'] = (df_test.loc[i, 'predicted'] == 'editor:close_case')    

In [ ]:
list(df_test['correct']).count(True)/len(list(df_test['correct']))

In [ ]:
def prediction(prev):
    return all_expected_events[1][prev]

df_test['time:between_predicted'] = df_test['concept:name'].apply(prediction)

In [ ]:
def time_difference(var):
    return abs((var[0] - var[1]).total_seconds())

df_test['time:predicton_off'] = df_test[['time:time_between', 'time:between_predicted']].apply(time_difference, axis=1)

In [ ]:
seconds = df_test['time:predicton_off'].mean()
minutes = seconds/60
hours = minutes/60
days = hours/24
print(str(seconds) + ', ' + str(minutes) + ', ' + str(hours) + ', ' + str(days))

In [ ]:
seconds = df_test['time:predicton_off'].median()
minutes = seconds/60
hours = minutes/60
days = hours/24
print(str(seconds) + ', ' + str(minutes) + ', ' + str(hours) + ', ' + str(days))

In [ ]:
145472/60/24